In [1]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import glob
import os
import pickle

In [31]:

url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
conn = sqlalchemy.create_engine(url)

In [32]:
# CONFS
schema = 'desenv.cam_' # PRODUÇÃO: 'a_camdep.'

In [6]:
query = """
DROP TABLE IF EXISTS {0}proposicoes;
CREATE TABLE {0}proposicoes AS
SELECT 
    DISTINCT ON(idproposicao)
    idproposicao, tipoproposicao, tema,
    nomeproposicao, ementa, explicacaoementa,
    autor, idecadastro, ufautor, partidoautor,
    dataapresentacao, regimetramitacao, 
    ultimodespacho, apreciacao, indexacao,
    situacao, linkinteiroteor, apensadas
FROM c_camdep.camdep_proposicoes_id
ORDER BY 
    idproposicao, id DESC;
""".format(schema)
conn.execute(query, autocommit=True)

In [7]:
query = """
ALTER TABLE {0}proposicoes
  ADD COLUMN ultimoorgao character varying(255);
""".format(schema)
conn.execute(query, autocommit=True)

In [8]:
query = """
ALTER TABLE {0}proposicoes
  ADD COLUMN status character varying(255);
""".format(schema)
conn.execute(query, autocommit=True)

In [17]:
query = """
ALTER TABLE {0}proposicoes
  ADD COLUMN resultado character varying(255);
""".format(schema)
conn.execute(query, autocommit=True)

In [14]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = split_part(situacao,' - ',1), 
    status = split_part(situacao,' - ',2)
WHERE 
    situacao LIKE '%%MESA - %%'
    AND ultimoorgao IS NULL
    AND status IS NULL
""".format(schema)
conn.execute(query)


UPDATE desenv.cam_proposicoes
SET
    ultimoorgao = split_part(situacao,' - ',1), 
    status = split_part(situacao,' - ',2)
WHERE 
    situacao LIKE '%%MESA -%%'
    AND ultimoorgao IS NULL
    AND status IS NULL



In [15]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = split_part(situacao,' - ',1), 
    status = split_part(situacao,' - ',2)
WHERE 
    situacao LIKE '%% - %%'
    AND ultimoorgao IS NULL
    AND status IS NULL
""".format(schema)
conn.execute(query)

In [16]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = 'Não identificado', 
    status = 'Não identificado'
WHERE 
    (
        situacao = 'Diversas' OR
        situacao IS NULL
    )
    AND ultimoorgao IS NULL
    AND status IS NULL
""".format(schema)
conn.execute(query)

In [19]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = 'Não identificado', 
    status = 'Aprovado',
    resultado = split_part(situacao,' no(a) ',2)
WHERE 
    ultimoorgao IS NULL
    AND status IS NULL
    AND situacao LIKE 'Tranformada n%%'
""".format(schema)
conn.execute(query)

In [20]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = 'Não identificado', 
    status = 'Originou outro ato',
    resultado = split_part(situacao,' o(a) ',2)
WHERE 
    ultimoorgao IS NULL
    AND status IS NULL
    AND situacao LIKE 'Originou %%'
""".format(schema)
conn.execute(query)

In [21]:
query = """
UPDATE {0}proposicoes
SET
    ultimoorgao = 'Não identificado', 
    status = 'Arquivada'
WHERE 
    ultimoorgao IS NULL
    AND status IS NULL
    AND situacao LIKE '%%Arquivada%%'
""".format(schema)
conn.execute(query)

In [34]:
query = """
COMMENT ON TABLE {0}proposicoes IS 
'NOVO COMENTÁRIO';
""".format(schema)
tmp = conn.execute(query)

AttributeError: 'NoneType' object has no attribute 'returning'